In [2]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model


In [3]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

In [5]:
## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('genger_label_encoder.pkl', 'rb') as file:
    gender_label_encoder = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [6]:
# Example input data
input_data = {
    'CreditScore': 800,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 38,
    'Tenure': 3,
    'Balance': 58000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [7]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Desktop\Projects\Churn_classification\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,Male,38,3,58000,2,1,1,60000


In [10]:
input_df['Gender']=gender_label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,1,38,3,58000,2,1,1,60000


In [11]:
## concatinat one hot encoded column
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,38,3,58000,2,1,1,60000,1.0,0.0,0.0


In [12]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.53956105,  0.91324755, -0.08535128, -0.69539349, -0.28983317,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [13]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 219ms/step


array([[0.00673263]], dtype=float32)

In [14]:
prediction_proba = prediction[0][0]

In [16]:
prediction_proba

0.0067326347

In [15]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
